<a href="https://colab.research.google.com/github/GabrielDeml/TumorClassifier/blob/main/TumorClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# %tensorflow_version 1.x

In [ ]:
!pip install tensorflowjs

     |████████████████████████████████| 71kB 7.9MB/s 
     |████████████████████████████████| 112kB 12.0MB/s 
  Found existing installation: tensorflow-hub 0.11.0
    Uninstalling tensorflow-hub-0.11.0:
      Successfully uninstalled tensorflow-hub-0.11.0


In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
from tensorflow.keras import datasets, layers, models

import IPython.display as display
import pathlib
import tensorflow as tf
import os
# tf.enable_eager_execution()
tf.__version__

import tensorflowjs as tfjs

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 32

In [ ]:
!mkdir .kaggle

In [ ]:
import json
token = {"username":"gabrieldeml","key":"3e37c76c042a65523596d5fdd5b719ae"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [ ]:
!chmod 600 /content/.kaggle/kaggle.json

In [ ]:
mkdir ~/.kaggle/

In [ ]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
!kaggle config set -n path -v{/content}

- path is now set to: {/content}


In [ ]:
#!kaggle competitions download -c dogs-vs-cats -p /content
!kaggle datasets download -d fanconic/skin-cancer-malignant-vs-benign -p /content

 97% 316M/325M [00:01<00:00, 245MB/s]
100% 325M/325M [00:01<00:00, 224MB/s]


In [ ]:
print ("skin-cancer-malignant-vs-benign: {}".format(os.path.exists("skin-cancer-malignant-vs-benign.zip")))
if not (os.path.exists("skin-cancer-malignant-vs-benign")):
  !unzip -q skin-cancer-malignant-vs-benign.zip
else:
  print("We already did this")

skin-cancer-malignant-vs-benign: True


In [ ]:
!mkdir train/cat
!mkdir train/dog
!mv train/cat* train/cat
!mv train/dog* train/dog

mv: cannot move 'train/cat' to a subdirectory of itself, 'train/cat/cat'
mv: cannot move 'train/dog' to a subdirectory of itself, 'train/dog/dog'


In [ ]:
data_root = pathlib.Path("data/train")

In [ ]:
for item in data_root.iterdir():
  print(item)

data/train/malignant
data/train/benign


In [ ]:
import random
all_image_paths = list(data_root.glob('*/*'))
all_image_paths = [str(path) for path in all_image_paths]
random.shuffle(all_image_paths)

image_count = len(all_image_paths)
image_count

2637

In [ ]:
label_names = sorted(item.name for item in data_root.glob('*/') if item.is_dir())
label_names

['benign', 'malignant']

In [ ]:
label_to_index = dict((name, index) for index,name in enumerate(label_names))
label_to_index

{'benign': 0, 'malignant': 1}

In [ ]:
all_image_labels = [label_to_index[pathlib.Path(path).parent.name]
                    for path in all_image_paths]

print("First 10 labels indices: ", all_image_labels[:10])

First 10 labels indices:  [0, 1, 1, 1, 0, 0, 0, 1, 1, 0]


In [ ]:
def preprocess_image(image):
  image = tf.image.decode_jpeg(image, channels=3)
  image = tf.image.resize(image, [224, 224])
  image /= 255.0  # normalize to [0,1] range

  return image

In [ ]:
def load_and_preprocess_image(path):
  image = tf.io.read_file(path)
  return preprocess_image(image)

In [ ]:
ds = tf.data.Dataset.from_tensor_slices((all_image_paths, all_image_labels))

# The tuples are unpacked into the positional arguments of the mapped function
def load_and_preprocess_from_path_label(path, label):
  return load_and_preprocess_image(path), label

image_label_ds = ds.map(load_and_preprocess_from_path_label)
image_label_ds

<MapDataset shapes: ((224, 224, 3), ()), types: (tf.float32, tf.int32)>

In [ ]:
ds = image_label_ds.apply(
tf.data.experimental.shuffle_and_repeat(buffer_size=10000))
ds = ds.batch(64)
ds = ds.prefetch(buffer_size=1000)
ds = ds.repeat()
ds
# ds = image_label_ds.apply(
#   tf.data.experimental.shuffle_and_repeat(buffer_size=image_count))
# ds = ds.batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)
# ds
# ds = ds.shuffle(buffer_size=10000) ## 10000: size of sample/record pool for random selection
# ds = ds.batch(32) ## 32: number of samples/records per batch (to be read into memory)
# ds = ds.repeat() ## None: keep repeating

Instructions for updating:
Use `tf.data.Dataset.shuffle(buffer_size, seed)` followed by `tf.data.Dataset.repeat(count)`. Static tf.data optimizations will take care of using the fused implementation.


<RepeatDataset shapes: ((None, 224, 224, 3), (None,)), types: (tf.float32, tf.int32)>

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(2, activation='softmax'))

In [ ]:
model.build()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 109, 109, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 52, 52, 64)        36928     
_________________________________________________________________
flatten (Flatten)            (None, 173056)            0         
_________________________________________________________________
dense (Dense)                (None, 64)                1

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=["accuracy"])

In [ ]:
model.fit(ds, epochs=600, steps_per_epoch=3)

Epoch 1/600
3/3 [==============================] - 19s 82ms/step - loss: 2.5818 - accuracy: 0.5749
Epoch 2/600
3/3 [==============================] - 0s 85ms/step - loss: 0.7801 - accuracy: 0.4453
Epoch 3/600
3/3 [==============================] - 0s 83ms/step - loss: 0.6579 - accuracy: 0.5443
Epoch 4/600
3/3 [==============================] - 0s 83ms/step - loss: 0.5696 - accuracy: 0.7096
Epoch 5/600
3/3 [==============================] - 0s 85ms/step - loss: 0.6518 - accuracy: 0.6374
Epoch 6/600
3/3 [==============================] - 0s 82ms/step - loss: 0.5741 - accuracy: 0.7161
Epoch 7/600
3/3 [==============================] - 0s 81ms/step - loss: 0.6739 - accuracy: 0.6693
Epoch 8/600
3/3 [==============================] - 0s 82ms/step - loss: 0.5414 - accuracy: 0.7585
Epoch 9/600
3/3 [==============================] - 0s 87ms/step - loss: 0.5372 - accuracy: 0.7194
Epoch 10/600
3/3 [==============================] - 0s 81ms/step - loss: 0.4905 - accuracy: 0.7318
Epoch 11/600
3/3 [

In [ ]:
tfjs.converters.save_keras_model(model, "saved_model")


/usr/local/lib/python3.7/dist-packages/tensorflowjs/converters/keras_h5_conversion.py:123: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  return h5py.File(h5file)


In [ ]:
!zip -r saved_model.zip saved_model/

  adding: saved_model/ (stored 0%)
  adding: saved_model/group1-shard2of11.bin (deflated 7%)
  adding: saved_model/group1-shard6of11.bin (deflated 7%)
  adding: saved_model/group1-shard10of11.bin (deflated 7%)
  adding: saved_model/group1-shard7of11.bin (deflated 7%)
  adding: saved_model/group1-shard4of11.bin (deflated 7%)
  adding: saved_model/group1-shard1of11.bin (deflated 7%)
  adding: saved_model/group1-shard3of11.bin (deflated 7%)
  adding: saved_model/group1-shard11of11.bin (deflated 7%)
  adding: saved_model/group1-shard9of11.bin (deflated 7%)
  adding: saved_model/group1-shard5of11.bin (deflated 7%)
  adding: saved_model/model.json (deflated 82%)
  adding: saved_model/group1-shard8of11.bin (deflated 7%)


In [ ]:
# !mkdir -p saved_model
# model.save('saved_model/model')

In [ ]:
# # Convert the model
# converter = tf.lite.TFLiteConverter.from_saved_model('/saved_model/model.pb') 
# # path to the SavedModel directory
# tflite_model = converter.convert()

# # Save the model.
# with open('model.tflite', 'wb') as f:
#   f.write(tflite_model)


In [ ]:
# # Convert the model.
# converter = tf.lite.TFLiteConverter.from_keras_model(model)
# tflite_model = converter.convert()

# # Save the model.
# with open('model.tflite', 'wb') as f:
#   f.write(tflite_model)
